# Notebook 3 - Tachycardia

Analyze signals for local tachycardia

In [ ]:
from multiprocessing import Pool, cpu_count
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import wfdb
from wfdb import processing

from vt.evaluate import calc_results
from vt.records import get_alarms, data_dir
from vt.preprocessing import fill_missing

In [ ]:
alarms, record_names, record_names_true, record_names_false = get_alarms()

## Section 0 - Extracting Tachycardia

In [ ]:
def has_tachycardia(qrs_0, qrs_1):
    """
    Use two sets of beat indices to determine whether or not
    tachycardia has occurred. Only return True if it occurs in
    both channels simultaneously.
    """
    if len(qrs_0) < 5 or len(qrs_1) < 5:
        return False
    
    # Iterate through groups of 5 channel 0 beats
    for qrs_num in range(len(qrs_0) - 4):
        local_beats_0 = qrs_0[qrs_num:qrs_num + 5]
        local_beats_1 = qrs_1[(qrs_1 > local_beats_0[0] - 40) & (qrs_1 < local_beats_0[-1] + 40)]
        
        # Too few beats
        if len(local_beats_1) < 5:
            return False
        
        # rr intervals
        rr = [np.diff(b) for b in [local_beats_0, local_beats_1]]        
        
        rr_mean = [np.mean(r) for r in rr]
        
        # Ventricular beat intervals must be consistent
        allowed_rr_deviation = [np.mean(r) + 2*np.std(r) for r in rr]
        for ch in range(2):
            if (np.min(rr[ch]) < rr_mean[ch] - allowed_rr_deviation[ch]
                    or np.max(rr[ch]) > rr_mean[ch] + allowed_rr_deviation[ch]):
                return False
        
        if (processing.calc_mean_hr(rr[0], fs=250) > 100
                and processing.calc_mean_hr(rr[1], fs=250) > 100):
            return True
    
    return False

In [ ]:
def calc_features(record_name):
    """
    Strategy is to find tachycardia for 5 beats in both of the ecg channels simultaneously.
    """
    fs = 250
    start_sec = 290
    stop_sec = 300
    # Read record
    signal, fields = wfdb.rdsamp(os.path.join(data_dir, record_name),
                                 sampfrom=start_sec * fs,
                                 sampto=stop_sec * fs, channels=[0,1,2])
    # Remove nans
    signal = fill_missing(signal)
    
    # Get beat indices
    qrs_0 = processing.gqrs_detect(signal[:, 0], fs=fs)
    qrs_1 = processing.gqrs_detect(signal[:, 1], fs=fs)
    
    # Figure out whether there is tachycardia
    tachycardia = has_tachycardia(qrs_0, qrs_1)
    
    # Alarm result
    result = alarms.loc[record_name, 'result']
    features = pd.DataFrame([[tachycardia, result]], columns=['tachycardia', 'result'], index=[record_name])
    return features
    

In [ ]:
# Calculate features for all records using multiple cpus
pool = Pool(processes=cpu_count() - 1)
features = pool.map(calc_features, record_names)

# Combine features into a single data frame
features = pd.concat(features)

print('Finished calculating features')

In [ ]:
display(features.head())

## Section 1 - Use tachycardia feature as sole input to classify alarms

In [ ]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features.iloc[:, :-1], features.iloc[:, -1],
                                                    train_size=0.75, test_size=0.25,
                                                    random_state=0)

# Note that we're not using y_train here because we are not using a supervised classifier

In [ ]:
# Calculate performance metrics
cm, p_correct, score = calc_results(y_true=y_test, y_pred=x_test['tachycardia'].values)


In [ ]:
# Display performance metrics
display(cm)
print('Final score:', score)

## How to deal with errors?

- Actual 0, predict 1 (false positives): Be more selective. Only accept ventricular tachycardia, rather than just tachycardia.
- Actual 1, predict 0 (false Negatives): Beat detector not working as well as intended, therefore cannot rely purely on beat based approach.

How to reconcile both simultaneously? Apply a non-beat specific approach which detects ventricular activity within a time window. Detecting ventricular activity should:
- Reduce pure tachycardia false positives. Incorrect misclassification should not increase FP.
- If used by itself, reduce missed beat false negatives. Incorrect misclassification may however increase false negatives. If used together with tachycardia, cannot reduce false negatives.